In [4]:
############################## Shoe detector
import cv2
import time
import sys
import numpy as np
sys.path.append("../modules/")
import input_output
import processor

def track_nothing(val):
    pass

def applySkel (img):
    res = cv2.ximgproc.thinning (img)
    return res

def applyBitwiseNot (img):
    res = cv2.bitwise_not (img)
    return res

det = processor.Processors ()
det.add_processor ("pict")
det.add_filter (processor.colorspace_to_colorspace("RGB", "HSV"), "pict", "rgb2hsv")
det.add_filter (processor.inrange ((30,13,0), (113,90,180)), "pict", "inrange")
det.add_filter (processor.morphology ("open", 0), "pict", "morph_open")
det.add_filter (processor.morphology ("close", 6), "pict", "morph_close")
det.add_filter (processor.morphology ("erode", 0), "pict", "morph_erode")
det.add_filter (processor.morphology ("dilate", 0), "pict", "morph_dilate")
det.add_filter (processor.custom_operation (applyBitwiseNot, "BitwiseNot"), "pict", "applyBitwiseNot")
det.add_filter (processor.filter_connected_components (), "pict", "conn")

#det.add_filter (processor.colorspace_to_colorspace("RGB", "GRAY"), "pict", "rgb2gray")
#det.add_filter (processor.custom_operation (applySkel, "skeletonize"), "pict", "skeletonize")

cv2.namedWindow ('trackbars', cv2.WINDOW_NORMAL)
cv2.createTrackbar ("thr_l_h", "trackbars", 30, 255, track_nothing)
cv2.createTrackbar ("thr_l_s", "trackbars", 13, 255, track_nothing)
cv2.createTrackbar ("thr_l_v", "trackbars", 0, 255, track_nothing)
cv2.createTrackbar ("thr_h_h", "trackbars", 113, 255, track_nothing)
cv2.createTrackbar ("thr_h_s", "trackbars", 90, 255, track_nothing)
cv2.createTrackbar ("thr_h_v", "trackbars", 180, 255, track_nothing)

cv2.createTrackbar ("morph_op", "trackbars", 0, 10, track_nothing)
cv2.createTrackbar ("morph_cl", "trackbars", 0, 10, track_nothing)
cv2.createTrackbar ("morph_er", "trackbars", 0, 10, track_nothing)
cv2.createTrackbar ("morph_di", "trackbars", 10, 10, track_nothing)

cv2.createTrackbar ("area_l", "trackbars", 90, 10000, track_nothing)
cv2.createTrackbar ("area_h", "trackbars", 180, 10000, track_nothing)

print ('Getting images...')
start = time.time()
try:
    source  = input_output.Source ("../data/images/log_video_frames/")
except:
    print ("Path not found")
    source.release()    
end = time.time()
print ('Duration {sec} seconds.'.format(sec = end - start))
while (True): 
    det.processors ["pict"] ["inrange"].set_ths (
        (cv2.getTrackbarPos ("thr_l_h", "trackbars"),
         cv2.getTrackbarPos ("thr_l_s", "trackbars"),
         cv2.getTrackbarPos ("thr_l_v", "trackbars")),
        (cv2.getTrackbarPos ("thr_h_h", "trackbars"),
         cv2.getTrackbarPos ("thr_h_s", "trackbars"),
         cv2.getTrackbarPos ("thr_h_v", "trackbars")))
    
    det.processors ["pict"] ["morph_open"].ker_sz = cv2.getTrackbarPos ("morph_op", "trackbars")
    det.processors ["pict"] ["morph_close"].ker_sz = cv2.getTrackbarPos ("morph_cl", "trackbars")
    det.processors ["pict"] ["morph_erode"].ker_sz = cv2.getTrackbarPos ("morph_er", "trackbars")
    det.processors ["pict"] ["morph_dilate"].ker_sz = cv2.getTrackbarPos ("morph_di", "trackbars")  
    
    det.processors ["pict"] ["conn"].area_low = cv2.getTrackbarPos ("area_l", "trackbars")  
    det.processors ["pict"] ["conn"].area_high = cv2.getTrackbarPos ("area_h", "trackbars")  
    
    
    frame = source.get_frame ()
    _,_ = det.process (frame, "pict")
  
    stages_picts = det.get_stages_picts ("pict")
    cv2.putText (stages_picts [0], str (source.files [source.file_num]), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 3)        
    cv2.imshow ("Picture", input_output.form_grid(stages_picts, window_x_sz=1200))

    key = cv2.waitKey (1)
    if key == 27:  # wait for ESC key to exit
         break
            
source.release()
cv2.destroyAllWindows()    

Getting images...
Duration 0.005995988845825195 seconds.
